In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter

In [2]:
max_words = 1000
random_state = 4

In [3]:
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\esimb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\esimb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\esimb\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [15]:
yield_df = pd.read_csv("../data/raw/yield_with_iso3.csv")
yield_df.drop(columns="Unnamed: 0", inplace= True)
yield_df.head(25)

,Area,Item,Year,hg/ha_yield,average_rain_fall_mm_per_year,pesticides_tonnes,avg_temp,Area_ISO3
0,Albania,Maize,1990,36613,1485.0,121.0,16.37,ALB
1,Albania,Potatoes,1990,66667,1485.0,121.0,16.37,ALB
2,Albania,"Rice, paddy",1990,23333,1485.0,121.0,16.37,ALB
3,Albania,Sorghum,1990,12500,1485.0,121.0,16.37,ALB
4,Albania,Soybeans,1990,7000,1485.0,121.0,16.37,ALB
5,Albania,Wheat,1990,30197,1485.0,121.0,16.37,ALB
6,Albania,Maize,1991,29068,1485.0,121.0,15.36,ALB
7,Albania,Potatoes,1991,77818,1485.0,121.0,15.36,ALB
8,Albania,"Rice, paddy",1991,28538,1485.0,121.0,15.36,ALB
9,Albania,Sorghum,1991,6667,1485.0,121.0,15.36,ALB


In [ ]:
categorical_cols = yield_df.select_dtypes(include=['object', 'category']).columns
print("Категориальные признаки:", categorical_cols.tolist())
yield_df.drop(columns=["Area"], inplace=True)
yield_df.head()


Категориальные признаки: ['Area', 'Item', 'Area_ISO3']


,Item,Year,hg/ha_yield,average_rain_fall_mm_per_year,pesticides_tonnes,avg_temp,Area_ISO3
0,Maize,1990,36613,1485.0,121.0,16.37,ALB
1,Potatoes,1990,66667,1485.0,121.0,16.37,ALB
2,"Rice, paddy",1990,23333,1485.0,121.0,16.37,ALB
3,Sorghum,1990,12500,1485.0,121.0,16.37,ALB
4,Soybeans,1990,7000,1485.0,121.0,16.37,ALB


In [20]:
yield_df["hg/ha_yield"] = yield_df["hg/ha_yield"] / 10000
yield_df.head()


,Item,Year,hg/ha_yield,average_rain_fall_mm_per_year,pesticides_tonnes,avg_temp,Area_ISO3
0,Maize,1990,3.6613,1485.0,121.0,16.37,ALB
1,Potatoes,1990,6.6667,1485.0,121.0,16.37,ALB
2,"Rice, paddy",1990,2.3333,1485.0,121.0,16.37,ALB
3,Sorghum,1990,1.2500,1485.0,121.0,16.37,ALB
4,Soybeans,1990,0.7000,1485.0,121.0,16.37,ALB


In [ ]:
def preprocess(text, stop_words, punctuation_marks, morph): #for the future
    tokens = word_tokenize(text.lower())
    preprocessed_text = []
    for token in tokens:
      if token not in punctuation_marks:
        lemma = morph.lemmatize(token)
        if lemma not in stop_words:
          preprocessed_text.append(lemma)
    return preprocessed_text


In [ ]:
punctuation_marks = ['!', ',', '(', ')', ':', '-', '?', '.', '..', '...', '/', '@', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '"']
morph = WordNetLemmatizer() 

In [ ]:
stop_words = stopwords.words("english")

In [23]:
yield_df = yield_df.rename(columns={"hg/ha_yield" : "t/ha_yield"})
yield_df.head(8)

,Item,Year,t/ha_yield,average_rain_fall_mm_per_year,pesticides_tonnes,avg_temp,Area_ISO3
0,Maize,1990,3.6613,1485.0,121.0,16.37,ALB
1,Potatoes,1990,6.6667,1485.0,121.0,16.37,ALB
2,"Rice, paddy",1990,2.3333,1485.0,121.0,16.37,ALB
3,Sorghum,1990,1.2500,1485.0,121.0,16.37,ALB
4,Soybeans,1990,0.7000,1485.0,121.0,16.37,ALB
5,Wheat,1990,3.0197,1485.0,121.0,16.37,ALB
6,Maize,1991,2.9068,1485.0,121.0,15.36,ALB
7,Potatoes,1991,7.7818,1485.0,121.0,15.36,ALB


In [25]:
yield_df.groupby("Item")["t/ha_yield"].describe()

,count,mean,std,min,25%,50%,75%,max
Item,,,,,,,,
Cassava,2045.0,15.047947,8.973852,1.1778,8.849200,12.82000,21.823200,38.5818
Maize,4121.0,3.631007,2.745637,0.0849,1.708600,2.54010,4.824300,20.7556
Plantains and others,556.0,10.604132,7.212798,2.1350,5.990075,8.98605,12.199800,41.8505
Potatoes,4276.0,19.980155,9.338218,0.8406,14.023325,18.22710,25.372900,50.1412
"Rice, paddy",3388.0,4.073043,1.916970,0.2034,2.731600,3.58780,5.001025,10.3895
Sorghum,3039.0,1.863578,1.511800,0.0578,0.778600,1.28850,2.625550,20.6000
Soybeans,3223.0,1.673109,0.760321,0.0050,1.073400,1.55330,2.207500,4.1609
Sweet potatoes,2890.0,11.905779,6.984757,0.8799,8.064300,9.99400,15.752650,40.0000
Wheat,3857.0,3.011627,1.838788,0.1706,1.869600,2.54970,3.644700,9.9387


In [35]:
yield_df.isin().count()

TypeError: DataFrame.isin() missing 1 required positional argument: 'values'

In [ ]:
X = yield_df.drop(columns=["t/ha_yield"])
y = yield_df["t/ha_yield"]



In [36]:
X.to_csv("../data/preprocessed/training.data.csv")
y.to_csv("../data/preprocessed/test.data.csv")

In [27]:
df.isnull().sum()
df.describe()

,soil_moisture_%,soil_pH,temperature_C,rainfall_mm,humidity_%,sunlight_hours,pesticide_usage_ml,yield_kg_per_hectare,latitude,longitude,region_le,croptype_le,cropdisease_le,irrigation_type_le,fertilizer_type_le,sowing_month,sowing_season
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.00000,500.000000,500.00000,500.000000,500.000000,500.000000,500.000000,500.000000,500.00000,500.000000,500.000000,500.000000
mean,26.750140,6.523980,24.675740,181.685740,65.194460,7.03014,26.586980,4032.92694,22.442473,80.392248,1.908000,1.934000,1.542000,1.56200,0.998000,2.062000,1.374000
std,10.150053,0.585558,5.348899,72.293091,14.642849,1.69167,13.202429,1174.43304,7.283492,5.910664,1.419707,1.424677,1.132608,1.08467,0.816903,0.826755,0.484348
min,10.160000,5.510000,15.000000,50.170000,40.230000,4.01000,5.050000,2023.56000,10.004243,70.020021,0.000000,0.000000,0.000000,0.00000,0.000000,1.000000,1.000000
25%,17.890000,6.030000,20.295000,119.217500,51.865000,5.66750,14.945000,2994.82000,16.263202,75.374713,1.000000,1.000000,0.750000,1.00000,0.000000,1.000000,1.000000
50%,25.855000,6.530000,24.655000,191.545000,65.685000,6.99500,25.980000,4071.69000,21.981743,80.650284,2.000000,2.000000,2.000000,2.00000,1.000000,2.000000,1.000000
75%,36.022500,7.040000,29.090000,239.035000,77.995000,8.47000,38.005000,5062.11000,28.528948,85.654629,3.000000,3.000000,3.000000,2.00000,2.000000,3.000000,2.000000
max,44.980000,7.500000,34.840000,298.960000,90.000000,10.00000,49.940000,5998.29000,34.981531,89.991901,4.000000,4.000000,3.000000,3.00000,2.000000,3.000000,2.000000
